# Import standard libraries

This analysis has taken inspiration from the research paper:
https://arxiv.org/pdf/1609.08399.pdf

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2

# Import dataset

The dataset comes in the following form:

HousesInfo.txt: 

Corresponding Images of the houses



Goal: Make the dataset in the following manner:


## Get housing Attributes

In [4]:
def get_house_attributes(iPath):
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(iPath, sep=" ", header=None, names=cols)
    return df

In [6]:
df = get_house_attributes("Houses Dataset/HousesInfo.txt")
df.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226


In [11]:
def get_images(df, iPath):
    images = []

    for i in df.index.values:
        basePath = os.path.sep.join([iPath, "{}_*".format(i + 1)])
        paths = sorted(list(glob.glob(basePath)))
        outImage = np.zeros((64,64,3), dtype="uint8")
        inImages = []

        for p in paths:
            image = cv2.imread(p)
            image = cv2.resize(image, (32, 32))
            inImages.append(image)

        # Tile images
        outImage[0:32, 0:32] = inImages[0]
        outImage[0:32, 32:64] = inImages[1]
        outImage[32:64, 0:32] = inImages[2]
        outImage[32:64, 32:64] = inImages[3]

        images.append(outImage)
    
    return np.array(images)
    

In [ ]:
images = get_images(df, "Houses Dataset/")

# Explaratory Data Analysis (EDA)

# Model Building

## Preprocessing

## Training

## Testing

# Conclusion